# Expirements 

In [28]:
# !pip install transformers datasets

In [2]:
from transformers import AutoTokenizer
from transformers import DataCollatorWithPadding
from transformers import TrainingArguments, Trainer
from transformers import AutoModelForSequenceClassification
from transformers import TrainingArguments, Trainer
from utils import getData
from datasets import load_dataset
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter
import seaborn as sns 
plt.style.use('ggplot')

## 1. HuggingFace

In [2]:
subTaskA = getData(sub_task = "A", return_type= "dataset")

Using custom data configuration default-c7c896b99937079e
Reusing dataset csv (/home/ahmad.shapiro/.cache/huggingface/datasets/csv/default-c7c896b99937079e/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519)


  0%|          | 0/2 [00:00<?, ?it/s]

### Subtask A

In [3]:
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True, max_length=512)

In [4]:
tokenizer = AutoTokenizer.from_pretrained("UBC-NLP/MARBERT")

In [5]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [6]:
tokenizedA = subTaskA.map(preprocess_function, batched=True)

Loading cached processed dataset at /home/ahmad.shapiro/.cache/huggingface/datasets/csv/default-c7c896b99937079e/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519/cache-7e5bbbfb8292748d.arrow


  0%|          | 0/2 [00:00<?, ?ba/s]

In [7]:
from datasets import load_metric
f1 = load_metric("f1")
recall = load_metric("recall")
precision =  load_metric("precision")

In [8]:
from transformers import EarlyStoppingCallback

In [9]:
def compute_metrics(p):    
    predictions, labels = p
    predictions = np.argmax(predictions, axis=1)
    metric = f1.compute(predictions=predictions, references=labels, average="macro")
    metric.update(recall.compute(predictions=predictions, references=labels, average="macro"))
    metric.update(precision.compute(predictions=predictions, references=labels, average="macro"))
    return metric

#### 1. MarBertV1 -- Dropout = 0 

In [11]:
model_mb1_d0 = AutoModelForSequenceClassification.from_pretrained("UBC-NLP/MARBERT", num_labels=2)

Some weights of the model checkpoint at UBC-NLP/MARBERT were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at U

In [12]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="./Checkpoints/MarbertV1_d0_lr_2e__5_t3",
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    evaluation_strategy="epoch",
    save_strategy = "epoch",
    metric_for_best_model="f1",
    num_train_epochs=20,
    weight_decay=0.01,
    load_best_model_at_end=True,
    group_by_length = True, 
    seed=2903
)

trainer = Trainer(
    model=model_mb1_d0,
    args=training_args,
    train_dataset=tokenizedA["train"],
    eval_dataset=tokenizedA["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    callbacks = [EarlyStoppingCallback(early_stopping_patience=3)]
)

trainer.train()

The following columns in the training set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
/home/ahmad.shapiro/anaconda3/envs/pytorch/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 8887
  Num Epochs = 20
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 4
  Gradient Accumulation steps = 1
  Total optimization steps = 44440


Epoch,Training Loss,Validation Loss,F1,Recall,Precision
1,0.582000,0.502906,0.817627,0.810246,0.827003
2,0.415400,0.817193,0.793413,0.810509,0.784894
3,0.217500,0.971773,0.799010,0.797050,0.801112
4,0.150400,1.027148,0.779744,0.788972,0.773441


The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1270
  Batch size = 4
Saving model checkpoint to ./Checkpoints/MarbertV1_d0_lr_2e__5_t3/checkpoint-2222
Configuration saved in ./Checkpoints/MarbertV1_d0_lr_2e__5_t3/checkpoint-2222/config.json
Model weights saved in ./Checkpoints/MarbertV1_d0_lr_2e__5_t3/checkpoint-2222/pytorch_model.bin
tokenizer config file saved in ./Checkpoints/MarbertV1_d0_lr_2e__5_t3/checkpoint-2222/tokenizer_config.json
Special tokens file saved in ./Checkpoints/MarbertV1_d0_lr_2e__5_t3/checkpoint-2222/special_tokens_map.json
The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertF

TrainOutput(global_step=8888, training_loss=0.35066804190566153, metrics={'train_runtime': 642.2976, 'train_samples_per_second': 276.725, 'train_steps_per_second': 69.189, 'total_flos': 458001833508660.0, 'train_loss': 0.35066804190566153, 'epoch': 4.0})

#### 2. MarBertV1 -- Dropout = 0.5

In [13]:
model_mb1_d_half = AutoModelForSequenceClassification.from_pretrained("UBC-NLP/MARBERT", num_labels=2, classifier_dropout=0.5)
training_args = TrainingArguments(
    output_dir="./Checkpoints/MarbertV1_d_half_lr_2e__5",
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    evaluation_strategy="epoch",
    save_strategy = "epoch",
    metric_for_best_model="f1",
    num_train_epochs=20,
    weight_decay=0.01,
    load_best_model_at_end=True,
    seed=2903
)

trainer = Trainer(
    model=model_mb1_d_half,
    args=training_args,
    train_dataset=tokenizedA["train"],
    eval_dataset=tokenizedA["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    callbacks = [EarlyStoppingCallback(early_stopping_patience=5)]
)

trainer.train()

Some weights of the model checkpoint at UBC-NLP/MARBERT were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at U

Epoch,Training Loss,Validation Loss,F1
1,0.564100,0.496552,0.739427
2,0.466000,0.704891,0.750663
3,0.241300,1.164276,0.731470
4,0.129800,1.156300,0.723735
5,0.089100,1.259465,0.716456
6,0.089900,1.448733,0.735739
7,0.068300,1.533938,0.720090


The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1266
  Batch size = 4
Saving model checkpoint to ./Checkpoints/MarbertV1_d_half_lr_2e__5/checkpoint-2140
Configuration saved in ./Checkpoints/MarbertV1_d_half_lr_2e__5/checkpoint-2140/config.json
Model weights saved in ./Checkpoints/MarbertV1_d_half_lr_2e__5/checkpoint-2140/pytorch_model.bin
tokenizer config file saved in ./Checkpoints/MarbertV1_d_half_lr_2e__5/checkpoint-2140/tokenizer_config.json
Special tokens file saved in ./Checkpoints/MarbertV1_d_half_lr_2e__5/checkpoint-2140/special_tokens_map.json
The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `

TrainOutput(global_step=14980, training_loss=0.23145715206742126, metrics={'train_runtime': 1191.0405, 'train_samples_per_second': 143.689, 'train_steps_per_second': 35.935, 'total_flos': 1368612751668660.0, 'train_loss': 0.23145715206742126, 'epoch': 7.0})

#### 1. MarBertV2 -- Dropout = 0 

In [13]:
tokenizer = AutoTokenizer.from_pretrained("/home/ahmad.shapiro/Research/Arabic Hate Speech 2022 shared task/Models/Marbertv2")

Downloading:   0%|          | 0.00/611 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/384 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/703k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.20M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [41]:
model_mb2_d0 = AutoModelForSequenceClassification.from_pretrained("/home/ahmad.shapiro/Research/Arabic Hate Speech 2022 shared task/Models/Marbertv2", num_labels=2)

Some weights of the model checkpoint at /home/ahmad.shapiro/Research/Arabic Hate Speech 2022 shared task/Models/Marbertv2 were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).

In [43]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="./Checkpoints/MarbertV2_d0_lr_2e__5",
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    evaluation_strategy="epoch",
    save_strategy = "epoch",
    metric_for_best_model="f1",
    num_train_epochs=100,
    weight_decay=0.01,
    load_best_model_at_end=True,
    seed=2903
)

trainer = Trainer(
    model=model_mb2_d0,
    args=training_args,
    train_dataset=tokenizedA["train"],
    eval_dataset=tokenizedA["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    callbacks = [EarlyStoppingCallback(early_stopping_patience=10)]
)

trainer.train()

The following columns in the training set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
/home/ahmad.shapiro/anaconda3/envs/pytorch/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 8557
  Num Epochs = 100
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 4
  Gradient Accumulation steps = 1
  Total optimization steps = 214000


Epoch,Training Loss,Validation Loss,F1
1,0.520500,0.435285,0.781095
2,0.357300,0.676497,0.759791
3,0.188500,0.816862,0.765152
4,0.100100,0.904044,0.762246
5,0.083000,1.111262,0.768683
6,0.109200,1.013478,0.775309
7,0.046500,1.347855,0.746386
8,0.049200,1.232313,0.775095
9,0.045500,1.304845,0.774118
10,0.014500,1.202215,0.751592


The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1266
  Batch size = 4
Saving model checkpoint to ./Checkpoints/MarbertV2_d0_lr_2e__5/checkpoint-2140
Configuration saved in ./Checkpoints/MarbertV2_d0_lr_2e__5/checkpoint-2140/config.json
Model weights saved in ./Checkpoints/MarbertV2_d0_lr_2e__5/checkpoint-2140/pytorch_model.bin
tokenizer config file saved in ./Checkpoints/MarbertV2_d0_lr_2e__5/checkpoint-2140/tokenizer_config.json
Special tokens file saved in ./Checkpoints/MarbertV2_d0_lr_2e__5/checkpoint-2140/special_tokens_map.json
The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClass

TrainOutput(global_step=23540, training_loss=0.14067806214776235, metrics={'train_runtime': 1699.9414, 'train_samples_per_second': 503.37, 'train_steps_per_second': 125.887, 'total_flos': 2147394239428920.0, 'train_loss': 0.14067806214776235, 'epoch': 11.0})

In [44]:
trainer.evaluate()

The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1266
  Batch size = 4


{'eval_loss': 0.4352850914001465,
 'eval_f1': 0.7810945273631842,
 'eval_runtime': 2.5425,
 'eval_samples_per_second': 497.942,
 'eval_steps_per_second': 124.682,
 'epoch': 11.0}

In [41]:
model_mb2_d0 = AutoModelForSequenceClassification.from_pretrained("/home/ahmad.shapiro/Research/Arabic Hate Speech 2022 shared task/Models/Marbertv2", num_labels=2)

Some weights of the model checkpoint at /home/ahmad.shapiro/Research/Arabic Hate Speech 2022 shared task/Models/Marbertv2 were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).

#### 1. MarBertV2 -- Dropout = 0.5 

In [10]:
from transformers import TrainingArguments, Trainer
model_mb2_d_half = AutoModelForSequenceClassification.from_pretrained("/home/ahmad.shapiro/Research/Arabic Hate Speech 2022 shared task/Models/Marbertv2", num_labels=2,  classifier_dropout=0.5)
training_args = TrainingArguments(
    output_dir="./Checkpoints/MarbertV2_d_half_lr_2e__5",
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    evaluation_strategy="epoch",
    save_strategy = "epoch",
    metric_for_best_model="f1",
    num_train_epochs=100,
    weight_decay=0.01,
    load_best_model_at_end=True,
    seed=2903
)

trainer = Trainer(
    model=model_mb2_d_half,
    args=training_args,
    train_dataset=tokenizedA["train"],
    eval_dataset=tokenizedA["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    callbacks = [EarlyStoppingCallback(early_stopping_patience=10)]
)

trainer.train()

Some weights of the model checkpoint at /home/ahmad.shapiro/Research/Arabic Hate Speech 2022 shared task/Models/Marbertv2 were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).

Epoch,Training Loss,Validation Loss,F1,Recall,Precision
1,0.536500,0.492904,0.833310,0.835745,0.831051
2,0.367500,0.675462,0.840296,0.834750,0.846819
3,0.239500,1.114862,0.817220,0.843513,0.807981
4,0.133700,0.947740,0.832193,0.825509,0.840378
5,0.080900,1.220089,0.819128,0.830883,0.811281
6,0.074700,1.232005,0.822921,0.822708,0.823136
7,0.048100,1.429694,0.816782,0.813879,0.819974
8,0.047000,1.300729,0.817232,0.821473,0.813551
9,0.019300,1.539938,0.797292,0.808526,0.789973
10,0.037900,1.531766,0.804965,0.817766,0.797001


The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1270
  Batch size = 4
Saving model checkpoint to ./Checkpoints/MarbertV2_d_half_lr_2e__5/checkpoint-2222
Configuration saved in ./Checkpoints/MarbertV2_d_half_lr_2e__5/checkpoint-2222/config.json
Model weights saved in ./Checkpoints/MarbertV2_d_half_lr_2e__5/checkpoint-2222/pytorch_model.bin
tokenizer config file saved in ./Checkpoints/MarbertV2_d_half_lr_2e__5/checkpoint-2222/tokenizer_config.json
Special tokens file saved in ./Checkpoints/MarbertV2_d_half_lr_2e__5/checkpoint-2222/special_tokens_map.json
The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `

TrainOutput(global_step=26664, training_loss=0.1323917897173924, metrics={'train_runtime': 1957.8047, 'train_samples_per_second': 453.927, 'train_steps_per_second': 113.494, 'total_flos': 2417603640507060.0, 'train_loss': 0.1323917897173924, 'epoch': 12.0})

### 3. AraBert V2

In [13]:
tokenizer = AutoTokenizer.from_pretrained("aubmindlab/bert-base-arabertv2")

Downloading:   0%|          | 0.00/611 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/384 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/703k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.20M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [ ]:
model_ab2_d_0 = AutoModelForSequenceClassification.from_pretrained("aubmindlab/bert-base-arabertv2", num_labels=2)
training_args = TrainingArguments(
    output_dir="./Checkpoints/ArabertV2_d_0",
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    evaluation_strategy="epoch",
    save_strategy = "epoch",
    metric_for_best_model="f1",
    num_train_epochs=100,
    weight_decay=0.01,
    load_best_model_at_end=True,
    seed=2903
)

trainer = Trainer(
    model=model_ab2_d_0,
    args=training_args,
    train_dataset=tokenizedA["train"],
    eval_dataset=tokenizedA["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    callbacks = [EarlyStoppingCallback(early_stopping_patience=10)]
)

trainer.train()

Downloading:   0%|          | 0.00/518M [00:00<?, ?B/s]

Some weights of the model checkpoint at aubmindlab/bert-base-arabertv2 were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were

Epoch,Training Loss,Validation Loss,F1
1,0.614200,0.495753,0.652068
2,0.595500,0.755050,0.652463
3,0.488700,0.971746,0.684642
4,0.386400,0.994983,0.688757
5,0.249700,1.235976,0.665241
6,0.207700,1.248299,0.692025
7,0.168400,1.526606,0.660955
8,0.150300,1.322744,0.667529
9,0.136000,1.570727,0.693683
10,0.092500,1.482610,0.681767


The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1266
  Batch size = 4
Saving model checkpoint to ./Checkpoints/ArabertV2_d_0/checkpoint-2140
Configuration saved in ./Checkpoints/ArabertV2_d_0/checkpoint-2140/config.json
Model weights saved in ./Checkpoints/ArabertV2_d_0/checkpoint-2140/pytorch_model.bin
tokenizer config file saved in ./Checkpoints/ArabertV2_d_0/checkpoint-2140/tokenizer_config.json
Special tokens file saved in ./Checkpoints/ArabertV2_d_0/checkpoint-2140/special_tokens_map.json
The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely igno

# With Pre-processing

## 1. HuggingFace

In [3]:
subTaskA = getData(sub_task = "A", return_type= "dataset", pre_proccessed=True)

Using custom data configuration default-6413a650cf0630eb
Reusing dataset csv (/home/ahmad.shapiro/.cache/huggingface/datasets/csv/default-6413a650cf0630eb/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519)


  0%|          | 0/2 [00:00<?, ?it/s]

### Subtask A

In [8]:
def preprocess_function(examples, tok):
    return tok(examples["text"], truncation=True, max_length=512)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
from datasets import load_metric
f1 = load_metric("f1")
recall = load_metric("recall")
precision =  load_metric("precision")
from transformers import EarlyStoppingCallback
def compute_metrics(p):    
    predictions, labels = p
    predictions = np.argmax(predictions, axis=1)
    metric = f1.compute(predictions=predictions, references=labels, average="macro")
    metric.update(recall.compute(predictions=predictions, references=labels, average="macro"))
    metric.update(precision.compute(predictions=predictions, references=labels, average="macro"))
    return metric

## Marbert V1 , d0 

In [6]:
tokenizer = AutoTokenizer.from_pretrained("UBC-NLP/MARBERT")

In [10]:
tokenizedA = subTaskA.map(preprocess_function,fn_kwargs = {'tok':tokenizer}, batched=True)

  0%|          | 0/9 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

In [8]:
model_mb1_pre_d0 = AutoModelForSequenceClassification.from_pretrained("UBC-NLP/MARBERT", num_labels=2)

Some weights of the model checkpoint at UBC-NLP/MARBERT were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at U

In [9]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="./Checkpoints/Prepro/MarbertV1_d0",
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    evaluation_strategy="epoch",
    save_strategy = "epoch",
    metric_for_best_model="f1",
    num_train_epochs=20,
    weight_decay=0.01,
    load_best_model_at_end=True,
    group_by_length = True, 
    seed=2903
)

trainer = Trainer(
    model=model_mb1_pre_d0,
    args=training_args,
    train_dataset=tokenizedA["train"],
    eval_dataset=tokenizedA["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    callbacks = [EarlyStoppingCallback(early_stopping_patience=3)]
)

trainer.train()

The following columns in the training set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
/home/ahmad.shapiro/anaconda3/envs/pytorch/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 8887
  Num Epochs = 20
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 4
  Gradient Accumulation steps = 1
  Total optimization steps = 44440


Epoch,Training Loss,Validation Loss,F1,Recall,Precision
1,0.568600,0.522857,0.797108,0.775173,0.846419
2,0.424400,0.893086,0.591630,0.601651,0.841909
3,0.230600,0.831189,0.786450,0.781455,0.792493
4,0.210700,1.033993,0.752953,0.763234,0.746710


The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1270
  Batch size = 4
Saving model checkpoint to ./Checkpoints/Prepro/MarbertV1_d0/checkpoint-2222
Configuration saved in ./Checkpoints/Prepro/MarbertV1_d0/checkpoint-2222/config.json
Model weights saved in ./Checkpoints/Prepro/MarbertV1_d0/checkpoint-2222/pytorch_model.bin
tokenizer config file saved in ./Checkpoints/Prepro/MarbertV1_d0/checkpoint-2222/tokenizer_config.json
Special tokens file saved in ./Checkpoints/Prepro/MarbertV1_d0/checkpoint-2222/special_tokens_map.json
The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.

TrainOutput(global_step=8888, training_loss=0.3759729993118025, metrics={'train_runtime': 577.5096, 'train_samples_per_second': 307.77, 'train_steps_per_second': 76.951, 'total_flos': 343885121577960.0, 'train_loss': 0.3759729993118025, 'epoch': 4.0})